In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms

In [2]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         # Initial block
#         self.conv1 = nn.Conv2d(1, 10, 3, padding=1)  # 28x28x10
#         self.bn1 = nn.BatchNorm2d(10)
       
#         # Block 1
#         self.conv2 = nn.Conv2d(10, 16, 3, padding=1)  # 28x28x16
#         self.bn2 = nn.BatchNorm2d(16)
#         self.conv3 = nn.Conv2d(16, 16, 3, padding=1)  # 28x28x16
#         self.bn3 = nn.BatchNorm2d(16)
       
#         # Transition 1
#         self.pool1 = nn.MaxPool2d(2, 2)  # 14x14x16
       
#         # Block 2
#         self.conv4 = nn.Conv2d(16, 16, 3, padding=1)  # 14x14x16
#         self.bn4 = nn.BatchNorm2d(16)
#         self.conv5 = nn.Conv2d(16, 16, 3, padding=1)  # 14x14x16
#         self.bn5 = nn.BatchNorm2d(16)
       
#         # Transition 2
#         self.pool2 = nn.MaxPool2d(2, 2)  # 7x7x16
       
#         # Final block
#         self.conv6 = nn.Conv2d(16, 16, 3)  # 5x5x16
#         self.bn6 = nn.BatchNorm2d(16)
#         self.conv7 = nn.Conv2d(16, 10, 1)  # 5x5x10
       
#         self.dropout = nn.Dropout(0.05)


#     def forward(self, x):
#         # Initial block
#         x = (F.relu(self.bn1(self.conv1(x))))
       
#         # Block 1
#         x = (F.relu(self.bn2(self.conv2(x))))
#         x = self.dropout(F.relu(self.bn3(self.conv3(x))))
#         x = self.pool1(x)
       
#         # Block 2
#         x = (F.relu(self.bn4(self.conv4(x))))
#         x = self.dropout(F.relu(self.bn5(self.conv5(x))))
#         x = self.pool2(x)
       
#         # Final block
#         x = self.dropout(F.relu(self.bn6(self.conv6(x))))
#         x = self.conv7(x)
#         x = F.adaptive_avg_pool2d(x, 1)
#         x = x.view(-1, 10)
#         return F.log_softmax(x, dim=1)


In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Initial Block
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False)  # 28x28x8
        self.bn1 = nn.BatchNorm2d(8)
        
        # Block 1
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1, bias=False)  # 28x28x16
        self.bn2 = nn.BatchNorm2d(16)
        
        # Transition 1
        self.pool1 = nn.MaxPool2d(2, 2)  # 14x14x16
        
        # Block 2
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1, bias=False)  # 14x14x16
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1, bias=False)  # 14x14x16
        self.bn4 = nn.BatchNorm2d(16)
        
        # Transition 2
        self.pool2 = nn.MaxPool2d(2, 2)  # 7x7x16
        
        # Block 3
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1, bias=False)  # 7x7x16
        self.bn5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 32, 3, padding=1, bias=False)  # 7x7x32
        self.bn6 = nn.BatchNorm2d(32)
        
        # Final Block
        self.conv7 = nn.Conv2d(32, 10, 1, bias=False)  # 7x7x10
        
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.dropout(self.bn1(F.relu(self.conv1(x))))
        
        x = self.dropout(self.bn2(F.relu(self.conv2(x))))
        x = self.pool1(x)
        
        x = self.dropout(self.bn3(F.relu(self.conv3(x))))
        x = self.dropout(self.bn4(F.relu(self.conv4(x))))
        x = self.pool2(x)
        
        x = self.dropout(self.bn5(F.relu(self.conv5(x))))
        x = self.dropout(self.bn6(F.relu(self.conv6(x))))
        
        x = self.conv7(x)
        x = F.adaptive_avg_pool2d(x, 1)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [27]:
# First install numpy
%pip install numpy

# Then install torchsummary (though you already have this)
%pip install torchsummary

# Now import and run your summary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,152
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,304
       BatchNorm2d-9           [-1, 16, 14, 14]              32
          Dropout-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           2,304
      BatchNorm2d-12           [-1, 16, 14, 14]              32
          Dropout-13           [-1, 16, 14, 14]               0
        MaxPool2d-14             [-1, 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
torch.manual_seed(1)
batch_size = 28


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=True, download=True,
#                     transform=transforms.Compose([
#                         transforms.ToTensor(),
#                         transforms.Normalize((0.5,), (0.5,))
#                     ])),
#     batch_size=batch_size, shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST('../data', train=False, transform=transforms.Compose([
#                         transforms.ToTensor(),
#                         transforms.Normalize((0.5,), (0.5,))
#                     ])),
#     batch_size=batch_size, shuffle=True, **kwargs)
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.RandomRotation((-15, 15)),
                       transforms.RandomAffine(0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=5),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)




In [29]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')




def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()


    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)


    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_acc))
   
    return test_loss, test_acc

In [31]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.1, epochs=20, steps_per_epoch=len(train_loader))
# scheduler = optim.lr_scheduler.OneCycleLR(
#     optimizer,
#     max_lr=0.1,
#     epochs=20,
#     steps_per_epoch=len(train_loader),
#     pct_start=0.2,  # Spend 20% of time in warmup
#     div_factor=10,  # Initial learning rate is max_lr/10
#     final_div_factor=100  # Final learning rate is initial_lr/100
# )

print("Starting Training...")
for epoch in range(1, 21):
    print(f"\nEpoch {epoch}/20:")
    train(model, device, train_loader, optimizer, epoch)
    test_loss, test_acc = test(model, device, test_loader)
   
    # Early stopping if we reach target accuracy
    if test_acc >= 99.4:
        print(f"\nReached target accuracy of {test_acc:.2f}% at epoch {epoch}")
        break



Starting Training...

Epoch 1/20:


loss=0.36201441287994385 batch_id=2142: 100%|██████████| 2143/2143 [01:04<00:00, 33.33it/s]  



Test set: Average loss: 0.0587, Accuracy: 9809/10000 (98.09%)


Epoch 2/20:


loss=0.033231236040592194 batch_id=2142: 100%|██████████| 2143/2143 [01:08<00:00, 31.23it/s] 



Test set: Average loss: 0.0444, Accuracy: 9855/10000 (98.55%)


Epoch 3/20:


...


Test set: Average loss: 0.0285, Accuracy: 9902/10000 (99.02%)


Epoch 9/20:


loss=0.059952590614557266 batch_id=2142: 100%|██████████| 2143/2143 [01:03<00:00, 33.94it/s] 



Test set: Average loss: 0.0294, Accuracy: 9909/10000 (99.09%)


Epoch 10/20:


loss=0.07877877354621887 batch_id=2142: 100%|██████████| 2143/2143 [01:03<00:00, 33.75it/s]  



Test set: Average loss: 0.0181, Accuracy: 9943/10000 (99.43%)


Reached target accuracy of 99.43% at epoch 10
